In [1]:
!pip install -q transformers langchain accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.1/289.1 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 539.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.5 MB/s eta 0:00:00


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [3]:
import torch
import json
from tqdm import tqdm
import transformers
from transformers import AutoTokenizer
from langchain import LLMChain, HuggingFacePipeline, PromptTemplate
from langchain.llms import VLLM

In [4]:
#import os
#os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "expandable_segments:True"

In [5]:
model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
def find_max_token_length(filename):
    with open(filename, 'r') as file:
        json_data = json.load(file)

    max_tokens = 0
    for record in json_data:
        knowledge_text = " ".join(record['knowledges'])  # Concatenate all knowledge statements
        tokenized_text = tokenizer.encode(knowledge_text, add_special_tokens=True)  # Tokenize the text
        token_count = len(tokenized_text)  # Count tokens
        if token_count > max_tokens:
            max_tokens = token_count

    return max_tokens

In [7]:
filename = '/content/concept_net.dev.csqa.json'
#max_tokens = find_max_token_length(filename)
#print(f"The maximum token length in the 'knowledges' field is: {max_tokens}")

In [8]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_length=2000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [9]:
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature': 0.4})

template = """
"Given the question below, identify and retain only the triples that are directly relevant or contribute to answering the question through connected reasoning. Remove any triples that do not help in answering the question.

Question: {question}
Triples: ```{text}```

Retained Triples:
"""

prompt = PromptTemplate(template=template, input_variables=["text", "question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [14]:
def process_records(filename):
    with open(filename, 'r') as file:
        json_data = json.load(file)

    for record in tqdm(json_data[1000:]):
        knowledge_text = " ".join(record['knowledges'])
        question_text = record['query']
        summary_result = llm_chain.run(text=knowledge_text, question=question_text)
        # Extract the summary part after "SUMMARY:"
        summary = summary_result.split("Retained Triples:")[1].strip() if "Retained Triples:" in summary_result else summary_result
        # Update the 'knowledges' field with the new summarized information
        record['knowledges'] = summary

    return json_data

In [15]:
def save_to_new_json(data, new_filename):
    with open(new_filename, 'w') as file:
        json.dump(data, file, indent=4)
    print(f"Data saved to {new_filename}")

filename = '/content/updated6_concept_net.dev.csqa.json'
new_filename = '/content/updated7_concept_net.dev.csqa.json'

# Process the records and save the updated data to a new file
updated_data = process_records(filename)
save_to_new_json(updated_data, new_filename)

  0%|          | 0/221 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
  0%|          | 1/221 [00:43<2:41:02, 43.92s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
  1%|          | 2/221 [01:16<2:16:23, 37.37s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
  1%|▏         | 3/221 [01:47<2:05:33, 34.56s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize

Data saved to /content/updated7_concept_net.dev.csqa.json
